In [ ]:
import os
from io import BytesIO
import openai                 
from datetime import datetime  
import base64      
import requests            
from PIL import Image         
import tkinter as tk          
from PIL import ImageTk       
import time

st = time.time()

def old_package(version, minimum):  # Block old openai python libraries before today's
    version_parts = list(map(int, version.split(".")))
    minimum_parts = list(map(int, minimum.split(".")))
    return version_parts < minimum_parts

if old_package(openai.__version__, "1.2.3"):
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the minimum version 1.2.3\n\n"
                     ">>You should run 'pip install --upgrade openai')")

from openai import OpenAI
client = OpenAI(api_key="[INSERT KEY]")

In [ ]:
#Edit this to whatever prompt you want
prompt = (
 "Subject: 14 number icon"  
 "Style: black and white pictogram"    
)

image_params = {
 "model": "dall-e-2",  
 "n": 1,           
 "size": "1024x1024",  
 "prompt": prompt,     # DALL-E 3: max 4000 characters, DALL-E 2: max 1000
 "user": "myName", 
 "response_format": "b64_json"
}

In [ ]:
# error handling
try:
    images_response = client.images.generate(**image_params)
except openai.APIConnectionError as e:
    print("Server connection error: {e.__cause__}")  # from httpx.
    raise
except openai.RateLimitError as e:
    print(f"OpenAI RATE LIMIT error {e.status_code}: (e.response)")
    raise
except openai.APIStatusError as e:
    print(f"OpenAI STATUS error {e.status_code}: (e.response)")
    raise
except openai.BadRequestError as e:
    print(f"OpenAI BAD REQUEST error {e.status_code}: (e.response)")
    raise
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    raise

# print time it took to generate image
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
# make a file name prefix from date-time of response
images_dt = datetime.utcfromtimestamp(images_response.created)
img_filename = images_dt.strftime('DALLE-%Y%m%d_%H%M%S')

In [ ]:
# save image
image_url_list = []
image_data_list = []
for image in images_response.data:
    image_url_list.append(image.model_dump()["url"])
    image_data_list.append(image.model_dump()["b64_json"])
image_objects = []

# Check whether lists contain urls that must be downloaded or b64_json images
if image_url_list and all(image_url_list):
    # Download images from the urls
    for i, url in enumerate(image_url_list):
        while True:
            try:
                print(f"getting URL: {url}")
                response = requests.get(url)
                response.raise_for_status()  # Raises stored HTTPError, if one occurred.
            except requests.HTTPError as e:
                print(f"Failed to download image from {url}. Error: {e.response.status_code}")
                retry = input("Retry? (y/n): ")  # ask script user if image url is bad
                if retry.lower() in ["n", "no"]:  # could wait a bit if not ready
                    raise
                else:
                    continue
            break
        image_objects.append(Image.open(BytesIO(response.content)))  # Append the Image object to the list
        image_objects[i].save(f"{img_filename}_{i}.png")
        print(f"{img_filename}_{i}.png was saved")
elif image_data_list and all(image_data_list):  # if there is b64 data
    # Convert "b64_json" data to png file
    for i, data in enumerate(image_data_list):
        image_objects.append(Image.open(BytesIO(base64.b64decode(data))))  # Append the Image object to the list
        image_objects[i].save(f"{img_filename}_{i}.png")
        print(f"{img_filename}_{i}.png was saved")
else:
    print("No image data was obtained. Maybe bad code?")